# Constants:

In [1]:
SEQ_LEN = 40 #240   # how many past candles to use to predict
CANDLES_SHIFT = 2 #5 # how many candles to shift between sequences
NAME = "4_m30_ov40th015p_shift2_seq40"
VALIDATION_PCT = 0.2

# Functions:

sequence split

In [2]:
import numpy as np

def splitDf_new(df):
    
    res = []
    print("")
    print("splitDf")
    while len(df) >= SEQ_LEN:
        first = df.head(SEQ_LEN).copy()
        first.index = np.arange(0, len(first))
        res.append(first)
        df = df.tail(len(df) - CANDLES_SHIFT)
        df.index = np.arange(0, len(df))

    print("-done")
    print("")
    return res

balance

In [3]:
def balance(dfs):
    
    buys = []
    sells = []
    holds = []
    for df in dfs:
        if df.at[len(df)-1, 'target'] == 0:
            sells.append(df)
        elif df.at[len(df)-1, 'target'] == 1:
            buys.append(df)
        else:
          holds.append(df)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    dfsBalanced = buys+sells+holds
    return dfsBalanced

preprocessing

In [4]:
from sklearn.preprocessing import StandardScaler


# Function to preprocess data
def preprocess3_train(df):
    # before sequencing
    #
    # log and pctchange transform price columns ('low', 'high', 'open', 'close')
    # scale every colum (center mean and unit variance)

    scaler_dict = {}
    for col in df.columns:
        if col != 'target':
            if col != 'quantity_baseUnits' and col != 'hl_percent':
                df[col] = np.log(df[col])
                df[col] = df[col].pct_change()
                df.dropna(inplace=True)
            scaler = StandardScaler()
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
            scaler_dict[col] = scaler
    df.index = np.arange(0, len(df))
    return df, scaler_dict

# Function to apply saved preprocessing to new data
def apply_preprocess3_val(df, scaler_dict):
    # before sequencing
    #
    # pct.change transform price columns ('low', 'high', 'open', 'close')
    # scale every colum (center mean and unit variance)
    
    for col in df.columns:
        if col != 'target':
            if col != 'quantity_baseUnits' and col != 'hl_percent':
                df[col] = np.log(df[col])
                df[col] = df[col].pct_change()
                df.dropna(inplace=True)
            scaler = scaler_dict[col]
            df[col] = scaler.transform(df[col].values.reshape(-1, 1))
    df.index = np.arange(0, len(df))
    return df

sequences

In [5]:
def buildSequences(dfs):
    
    sequences = []
    for df in dfs:
        if(len(df) == SEQ_LEN):
            label = df.at[SEQ_LEN-1, 'target']
            df = df.iloc[:, :-1]
            dfArray = df.values.tolist()
            sequences.append([np.array(dfArray), label])
    
    return sequences

X y split

In [6]:
def xySplit(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)

# DF manipulation, build training sets:

In [7]:
import pandas as pd
df = pd.read_csv("../historicalData/labeled/HistoricalDataLabeled_BTC_USDT_01072016_01072023_MINUTE_30_ov40_th015p.csv")
df = df[['close', 'weightedAverage', 'hl_percent', 'quantity_baseUnits', 'target']]
df

,close,weightedAverage,hl_percent,quantity_baseUnits,target
0,678.86,680.43,0.006774,10.651774,0
1,678.54,678.68,0.005044,6.966340,0
2,681.60,679.31,0.004811,9.486937,0
3,675.00,677.94,0.011018,14.400582,2
4,670.73,671.68,0.006326,35.396886,2
...,...,...,...,...,...
122671,30348.04,30379.52,0.003004,2.116099,0
122672,30388.37,30364.50,0.002573,1.236853,0
122673,30406.47,30403.91,0.001523,0.688637,0
122674,30426.45,30411.08,0.000850,0.838506,0


In [8]:
# Split data into train and validation sets
train_size = int((1-VALIDATION_PCT) * len(df))
train_df = df.iloc[:train_size].copy()
val_df = df.iloc[train_size:].copy()

In [9]:
# Preprocess the training data and save the scaling parameters
train_df, scaler_dict = preprocess3_train(train_df)
# Apply saved preprocessing to validation data
val_df = apply_preprocess3_val(val_df, scaler_dict)

split into dfs with SEQ_LEN rows

In [10]:
train_dfs = splitDf_new(train_df)
val_dfs = splitDf_new(val_df)


splitDf
-done


splitDf
-done



balance buys/sells/holds

In [11]:
# balance buys, sells, and holds
import random

random.shuffle(train_dfs)
random.shuffle(val_dfs)
train_dfs = balance(train_dfs)
val_dfs = balance(val_dfs)
random.shuffle(train_dfs)
random.shuffle(val_dfs)

before balancing:
buys: 16439 , sells: 15081 , holds: 17530
before balancing:
buys: 3667 , sells: 3759 , holds: 4822


build sequences

In [12]:
sequencesTraining = buildSequences(train_dfs)
sequencesValidation = buildSequences(val_dfs)
random.shuffle(sequencesTraining)
random.shuffle(sequencesValidation)

split sequence from label

In [13]:
train_x, train_y = xySplit(sequencesTraining)
validation_x, validation_y = xySplit(sequencesValidation)

In [14]:
print(type(train_x))
print(train_x.shape)
print(type(validation_x))
print(validation_x.shape)

<class 'numpy.ndarray'>
(45243, 40, 4)
<class 'numpy.ndarray'>
(11001, 40, 4)


In [15]:
import pickle

with open(f'../trainData/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'../trainData/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'../trainData/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'../trainData/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [2]:
import pickle

with open(f"../trainData/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"../trainData/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"../trainData/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"../trainData/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [4]:
EPOCHS = 400

batchSize = 96
layers = 2
nodes = 64#256
denseNodes = 32#128

dropOut = 0.4#0.8 #0.92 #0.88
rec_dropout = 0
l1l2_reg = 0 #1e-5#1e-3

learningRate = 0.00001
decay = 0

hp_suffix = f"bs({batchSize})_layers({layers})_noded({nodes})_dNodes({denseNodes})_do({int(dropOut*100)}%)_recdo({rec_dropout})_l1l2({l1l2_reg})_lr({learningRate})_decay({decay})"

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
import json
# model
model = Sequential()

for _ in range(layers-1):
  model.add(Bidirectional(LSTM(nodes, 
               activation="tanh", 
               recurrent_activation = 'sigmoid', 
               recurrent_dropout = rec_dropout, 
               unroll = False, 
               use_bias = True, 
               input_shape=(train_x.shape[1:]), 
               return_sequences=True,
               kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
               #bias_regularizer=regularizers.l2(l1l2_reg),
               activity_regularizer=regularizers.l2(l1l2_reg)
               )))
  model.add(Dropout(dropOut))
  #model.add(BatchNormalization())

model.add(Bidirectional(LSTM(nodes, 
             activation="tanh", 
             recurrent_activation = 'sigmoid', 
             recurrent_dropout = rec_dropout, 
             unroll = False, 
             use_bias = True, 
             input_shape=(train_x.shape[1:]),
             kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
             #bias_regularizer=regularizers.l2(l1l2_reg),
             activity_regularizer=regularizers.l2(l1l2_reg)
             )))
model.add(Dropout(dropOut))
#model.add(BatchNormalization())

model.add(Dense(denseNodes, 
                activation="relu",
                kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                #bias_regularizer=regularizers.l2(l1l2_reg), 
                activity_regularizer=regularizers.l2(l1l2_reg)))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))



# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])


filename = "" + NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

# Prepare TensorBoard callback
log_dir = "../logs/fit/" + f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# store hyper-parameters
# Path to the JSON file
file_path = "models/hps/hps.json"
# Load the JSON file into a dictionary
with open(file_path, "r") as json_file:
    data = json.load(json_file)
# Add a new key-value pair to the dictionary
data[f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")] = hp_suffix
# Write the updated dictionary back to the file
with open(file_path, "w") as json_file:
    json.dump(data, json_file, indent=4)




# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])
  #callbacks=[checkpoint])

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/400 [00:00<?, ?epoch/s]

Epoch 1/400
469/472 [============================>.] - ETA: 0s - loss: 1.0975 - accuracy: 0.3575

  0%|          | 1/400 [00:07<50:09,  7.54s/epoch, loss=1.1, accuracy=0.358, val_loss=1.09, val_accuracy=0.423]

472/472 [==============================] - 8s 9ms/step - loss: 1.0974 - accuracy: 0.3576 - val_loss: 1.0939 - val_accuracy: 0.4226
Epoch 2/400
472/472 [==============================] - ETA: 0s - loss: 1.0906 - accuracy: 0.3939

...

 70%|███████   | 281/400 [17:56<08:06,  4.09s/epoch, loss=0.999, accuracy=0.484, val_loss=1.01, val_accuracy=0.461]

472/472 [==============================] - 3s 7ms/step - loss: 0.9994 - accuracy: 0.4838 - val_loss: 1.0142 - val_accuracy: 0.4614
Epoch 282/400
467/472 [============================>.] - ETA: 0s - loss: 0.9986 - accuracy: 0.4836

 70%|███████   | 282/400 [18:00<07:44,  3.94s/epoch, loss=0.999, accuracy=0.483, val_loss=1.01, val_accuracy=0.458]

472/472 [==============================] - 4s 8ms/step - loss: 0.9988 - accuracy: 0.4833 - val_loss: 1.0150 - val_accuracy: 0.4584
Epoch 283/400
371/472 [======================>.......] - ETA: 0s - loss: 0.9993 - accuracy: 0.4876

KeyboardInterrupt: 